In [1]:

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import glob
import os
from pdb import set_trace as stop
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage import median_filter
from scipy.ndimage import label
from matplotlib import cm
from scipy import ndimage
import random
import scipy
import pickle
import datetime
import pandas as pd
import subprocess
import matplotlib.path as mplPath
import sys
import calendar
from calendar import monthrange
import warnings
warnings.filterwarnings("ignore")
from itertools import groupby

#### speed up interpolation
import scipy.interpolate as spint
import scipy.spatial.qhull as qhull
import numpy as np
import xarray as xr
import netCDF4

#### Functions from "Tracking_Function.py" file
from Tracking_Functions import ObjectCharacteristics
from Tracking_Functions import interp_weights
from Tracking_Functions import interpolate
from Tracking_Functions import detect_local_minima
from Tracking_Functions import Feature_Calculation
from Tracking_Functions import haversine
from Tracking_Functions import BreakupObjects
from Tracking_Functions import ConnectLon
from Tracking_Functions import ReadERA5
from Tracking_Functions import minimum_bounding_rectangle 
from Tracking_Functions import is_land 
from Tracking_Functions import DistanceCoord 
from Tracking_Functions import MultiObjectIdentification

from importlib import reload
import MultiObjectIdentification_noBT
reload(MultiObjectIdentification_noBT)
from MultiObjectIdentification_noBT import MultiObjectIdentification_noBT

def remap_feature_tracking(flnm, DataName, sTarGrid, Lat, Lon, rgrGridCells, dir_o, var):
    '''
    This is from Andy's original EURO_CORDEX_feature_tracking
              calculating interp_weights
    flnm:input file name
    DataName: dataset of flnm
    sTarGrid: name of target grid
    Lat, Lon: Target grid lat and lon
    var:      variable name. Becasue for some models, different variables are on different grids
    return: vtxE, wtsE
    '''
    
    flnm_o = dir_o + 'Regrid_'+DataName+'_to_'+sTarGrid+'_' + var + '.npz'
    print('remaping file', flnm_o)
    
    if os.path.isfile(flnm_o) == False:
        # RCM MOHC-HadREM3-GA7-05 use 'latitude', 'longitude' as coordinate
        if 'MOHC-HadREM3-GA7-05' or 'CERRA' in DataName:
            COORDINATES = [flnm, 'latitude', 'longitude']
        else:
            COORDINATES = [flnm, 'lat', 'lon']
 
        # read in the coordinates
        ncid=Dataset(COORDINATES[0], mode='r')
        LatM=np.squeeze(ncid.variables[COORDINATES[1]])
        LonM=np.squeeze(ncid.variables[COORDINATES[2]])
        ncid.close()

        if np.max(Lon) > 180:
            Lon[Lon > 180] = Lon[Lon > 180] - 360

        # Remap Model to Target grid
        points = np.array([LonM.flatten(), LatM.flatten()]).transpose()
        vtxE, wtsE = interp_weights(points, np.append(Lon.flatten()[:,None], Lat.flatten()[:,None], axis=1))
        np.savez(flnm_o, vtxE=vtxE, wtsE=wtsE)
    else:
        with  np.load(flnm_o) as DATA:
            vtxE=DATA['vtxE']
            wtsE=DATA['wtsE']

    return vtxE, wtsE


In [2]:
####  USER MODIFY SECTION
### regrid to .44 or .11
res_TarGrid = 11
res  = 11

OutputFolder='/glade/campaign/mmm/c3we/mingge/FRONTIERS/data/ProcessIdentification/CORDEX/TarGrid.'+ str(res_TarGrid)+'/EUR-' + str(res)
# create output directory if it does not exists already
if not os.path.exists(OutputFolder):
    os.makedirs(OutputFolder)
    
FocusRegion = [67.92, 50.06, 27, -32] #  N, E, S, W
Variables = ['V', 'U', 'T', 'Q', 'SLP','IVTE','IVTN']
VarPL = [850,850,850,850,-1,-1,-1] # pressure level of variable in hPa; put 2D for surface variable


# target grid is generated by - ~/projects/FRONTIER/programs/CommonGrids/CommonGrids.ipynb
TargetGrid = '/glade/u/home/prein/projects/FRONTIER/programs/CommonGrids/Regular_0.' + str(res_TarGrid) +'_EURO-CORDEX.nc'
sTarGrid ='EURO0' + str(res_TarGrid) + 'i'

dT = 3  # temporal resolution in hours

In [3]:
# precipitation tracking options
SmoothSigmaP = 1  # Gaussion std for precipitation smoothing
Pthreshold = 2    # precipitation threshold [mm/h]
MinTimePR = 4     # in hours
MinAreaPR = 5000 #2000 # km2

# minimum Moisture Stream 
MinTimeMS = 9 # in hours
# MinVolMS = MinTimeMS*10*10 # hours x lon x lat grid cells
MinAreaMS = 100000 # km2
MinMSthreshold = 0.13 # 1 was default # g*m/g*s

# cyclone tracking
MinTimeCY = 12 # minimum livetime in hours
MaxPresAnCY = -8 # hPa

# anty cyclone tracking
MinTimeACY = MinTimeCY # minimum livetime in hours
MinPresAnACY = 6 # hPa

# Frontal zones
MinAreaFR = 50000 # km2

# Cloud tracking setup
SmoothSigmaC = 1
Cthreshold = 241
MinTimeC = 9 # in hours
MinAreaC = 40000 # km

# AR tracking
IVTtrheshold = 500 # [kg m-1 s-1]
MinTimeIVT = 9 # hours
AR_MinLen = 2000 # km
AR_Lat = 20 # AR centroids have to be poeward of this latitude
AR_width_lenght_ratio = 2

# TC detection
TC_Pmin = 995 #995
TC_lat_genesis = 35 #35  # absolute latitude
TC_lat_max = 60 # 60 maximum latitue of a TC
TC_deltaT_core = 0 #0  degree celsius
TC_T850min = 285 # 285 minimum temperature of TC core at 850hPa
TC_minBT = 241 # 220 K --> maximum average cloud top brightness temperature

# MCs detection
MCS_Minsize = 5000 #km 
MCS_minPR = 15 # minimum max precipitation in mm/h
CL_MaxT = 215 # minimum brightness temperature
CL_Area = 40000 # min cloud area size in km2
MCS_minTime = 4 # minimum time step (here 3-hours)

In [4]:
# ### Read the target grid
ncid=Dataset(TargetGrid, mode='r')
Lat=np.squeeze(ncid.variables['lat'][:])
Lon=np.squeeze(ncid.variables['lon'][:])
Mask=np.squeeze(ncid.variables['Mask'][:])
# Zfull=np.squeeze(ncid.variables['Z'][:])
ncid.close()

Lon[Lon >= 180] = Lon[Lon >= 180]-360
if len(Lon.shape) == 1:
    Lon,Lat = np.meshgrid(Lon,Lat)

# if (FocusRegion[1] > 0) & (FocusRegion[3] < 0):
#     # region crosses zero meridian
#     iRoll = np.sum(Lon[0,:] < 0)
# else:
#     iRoll=0
# Lon = np.roll(Lon,iRoll, axis=1)
iNorth = np.argmin(np.abs(Lat[:,0] - FocusRegion[0]))
iSouth = np.argmin(np.abs(Lat[:,0] - FocusRegion[2]))+1
iEeast = np.argmin(np.abs(Lon[0,:] - FocusRegion[1]))+1
iWest = np.argmin(np.abs(Lon[0,:] - FocusRegion[3]))

print('iNorth,iSouth,iWest,iEeast:', iNorth,iSouth,iWest,iEeast)

Lon = Lon[iSouth:iNorth,iWest:iEeast]
Lat = Lat[iSouth:iNorth,iWest:iEeast]
Mask = Mask[iSouth:iNorth,iWest:iEeast]

rgrGridCells=[(Lon.ravel()[ii],Lat.ravel()[ii]) for ii in range(len(Lon.ravel()))]

iNorth,iSouth,iWest,iEeast: 372 1 0 747


In [5]:
DataDir = '/glade/campaign/mmm/c3we/FRONTIER/data/EUR-'+ str(res) + '/'
DataDir = '/glade/campaign/mmm/c3we/CERRA/3hr/'
NeededVars = ['V', 'U', 'T', 'Q', 'SLP', 'IVTE', 'IVTN', 'PR']
VARS = ['va850', 'ua850', 'ta850', 'hus850', 'psl', '','','pr']
    
# Ming Ge
## for experiment 
DataName = 'CERRA'

dir_o = OutputFolder + '/CERRA/'

print(DataName)
print('DataDir', DataDir)
print('dir_o', dir_o)

if not os.path.exists(dir_o):
    os.makedirs(dir_o)
    print('not exist and create', dir_o)


CERRA
DataDir /glade/campaign/mmm/c3we/CERRA/3hr/
dir_o /glade/campaign/mmm/c3we/mingge/FRONTIERS/data/ProcessIdentification/CORDEX/TarGrid.11/EUR-11/CERRA/
not exist and create /glade/campaign/mmm/c3we/mingge/FRONTIERS/data/ProcessIdentification/CORDEX/TarGrid.11/EUR-11/CERRA/


In [6]:
rgrGridCells[:2]

[(-32.0, 27.11), (-31.89, 27.11)]

In [ ]:
year_s = 1985
year_e = 2020

#for yy in range(year_s, year_e + 1):
for yy in range(year_s+1, year_e + 1):
    StartDay = datetime.datetime(yy, 1, 1, 0) 
    StopDay  = datetime.datetime(yy, 12, 31, 23)
    Time = pd.date_range(StartDay, end=StopDay, freq=str(dT)+'h')
    
    DATA_all = np.zeros((len(Time), Lon.shape[0], Lon.shape[1], len(VARS)))
    DATA_all[:] = np.nan
    
    is_all = True
    
    for va in range(len(VARS)):
    #for va in range(1):
        print('VARS[va]',VARS[va])
        if VARS[va] != '':
            FILE = DataDir + VARS[va] +'/'+VARS[va] + '_CERRA_3hr_' + str(yy) + '.nc'
            
            if not FILE:
                print(yy, DataName, 'data does not exist')
                is_all = False
                break
                
            print('FILE', FILE)
            vtxE, wtsE = remap_feature_tracking(FILE, DataName, sTarGrid, Lat, Lon, rgrGridCells, dir_o, VARS[va])
            
            ncid = Dataset(FILE, mode='r')
            time_nc = ncid.variables['time']
            TIME_nc = np.squeeze(time_nc)
            DayFrac = TIME_nc[1]-TIME_nc[0] 
             
            # num2date datetime objects cftime.DatetimeGregorian
            dates = netCDF4.num2date(time_nc, units = time_nc.units, calendar=time_nc.calendar) 
            date_ss = datetime.datetime(dates[0].year,  dates[0].month,  dates[0].day,  dates[0].hour,  dates[0].minute)
            date_ee = datetime.datetime(dates[-1].year, dates[-1].month, dates[-1].day, dates[-1].hour, dates[-1].minute)
            
            if DayFrac == 0.25 or DayFrac == 6 :
                TimeAct = pd.date_range(date_ss, end=date_ee, freq='6h')
                if len(TIME_nc)==1460 and calendar.isleap(yy) and dates[-1].day == 31:
                    # without a leap day
                    TimeAct = TimeAct[(TimeAct.day != 29) | (TimeAct.month != 2)]
            elif DayFrac == 0.125 or DayFrac == 3:   
                # for 3-hr Precipitation: put 1:30 -> 00 to avoid interpolation
                if  dates[0].hour == 1 and dates[0].minute== 30 :
                    date_ss= datetime.datetime(dates[0].year, dates[0].month, dates[0].day, 0, 0)
                if  dates[-1].hour == 22 and dates[-1].minute== 30 :
                    date_ee = datetime.datetime(dates[-1].year, dates[-1].month, dates[-1].day, 21, 0)
                TimeAct = pd.date_range(date_ss, end=date_ee, freq='3h')
                if len(TIME_nc)==2920 and calendar.isleap(yy) and dates[-1].day == 31:
                        TimeAct = TimeAct[(TimeAct.day != 29) | (TimeAct.month != 2)]
            else:
                print('DayFrac:', DayFrac,  'it should be 0.25/6 or 0.125/3')
                exit()
                        
            # for annual run, we set TimeActMM and TimeAc the same
            TimeActMM = TimeAct 
            
            iTT = np.isin(TimeAct,Time)
            iTTMM = np.where(np.isin(Time,TimeActMM) == True)[0]          
            
            if VARS[va] != 'pr':
                DATAact = np.squeeze(ncid.variables[VARS[va]][iTT,:,:])
            else:
                # precipitation from kg/m2/s to mm/hr
                DATAact = np.squeeze(ncid.variables[VARS[va]][iTT,:,:])*3600*3
            ncid.close()
             
            for tt in range(DATAact.shape[0]):
                valuesi=interpolate(DATAact[tt,:,:].flatten(), vtxE, wtsE)
                DATA_all[iTTMM[tt],:,:,va] = valuesi.reshape(Lat.shape[0],Lat.shape[1])
                 
    if is_all == True:
        # set data to NaN outside masked region
        DATA_all[:,Mask == 0,:] = np.nan
        #print('0',DATA_all[0,:,:,0].max())
        #print('1',DATA_all[0,:,:,1].max())
        # translate upward longwave radiation to brigthness temperature
        # WRF LWUPT to BT
        #OLR_hourly = DATA_all[:,:,:, NeededVars.index('BT')]
    
        # transfore OLR to BT using: https://link.springer.com/article/10.1007/s11434-011-4686-6
        #Tf = (OLR_hourly/5.6693e-8)**(1./4.)
        #a = -0.000917
        #b =  1.13333
        #c =  10.50007
        #c = c - Tf
    
        #DATA_all[:,:,:,NeededVars.index('BT')] = (-b+np.sqrt(b**2-4*a*c))/(2.*a)

        MultiObjectIdentification_noBT(DATA_all,
                         Lon,
                         Lat,
                         Time,
                         dT,
                         Mask,
                         OutputFolder = dir_o,
                         DataName = DataName)
    

VARS[va] va850
FILE /glade/campaign/mmm/c3we/CERRA/3hr/va850/va850_CERRA_3hr_1986.nc
remaping file /glade/campaign/mmm/c3we/mingge/FRONTIERS/data/ProcessIdentification/CORDEX/TarGrid.11/EUR-11/CERRA/Regrid_CERRA_to_EURO011i_va850.npz
VARS[va] ua850
FILE /glade/campaign/mmm/c3we/CERRA/3hr/ua850/ua850_CERRA_3hr_1986.nc
remaping file /glade/campaign/mmm/c3we/mingge/FRONTIERS/data/ProcessIdentification/CORDEX/TarGrid.11/EUR-11/CERRA/Regrid_CERRA_to_EURO011i_ua850.npz
VARS[va] ta850
FILE /glade/campaign/mmm/c3we/CERRA/3hr/ta850/ta850_CERRA_3hr_1986.nc
remaping file /glade/campaign/mmm/c3we/mingge/FRONTIERS/data/ProcessIdentification/CORDEX/TarGrid.11/EUR-11/CERRA/Regrid_CERRA_to_EURO011i_ta850.npz
VARS[va] hus850
FILE /glade/campaign/mmm/c3we/CERRA/3hr/hus850/hus850_CERRA_3hr_1986.nc
remaping file /glade/campaign/mmm/c3we/mingge/FRONTIERS/data/ProcessIdentification/CORDEX/TarGrid.11/EUR-11/CERRA/Regrid_CERRA_to_EURO011i_hus850.npz
VARS[va] psl
FILE /glade/campaign/mmm/c3we/CERRA/3hr/psl/psl